In [20]:
import os
import re
import cv2
from skimage.metrics import structural_similarity
import matplotlib.pyplot as plt

In [2]:
root_data_folder = "/projects/SkyGAN/clouds_fisheye"
desc_file = "processed_1K_JPGs.txt"

In [3]:
# Get all paths to .jpg images
def get_paths_list(root_data_folder, desc_file):
    file_path = os.path.join(root_data_folder, desc_file)

    with open(file_path) as f:
        paths = f.read().strip().split('\n')
        paths = list(map(lambda x: os.path.join(root_data_folder, x), paths))

    return paths

In [4]:
paths_list = get_paths_list(root_data_folder, desc_file)
f"Number of images: {len(paths_list)}"

'Number of images: 18840'

In [5]:
# Separating images in different folders
def image_paths_to_dict(paths):
    locations_map = {}
    for path in paths:
        full_path, file = os.path.split(path)
        if full_path in locations_map:
            locations_map[full_path].append(file)
        else:
            locations_map[full_path] = [file]
    return locations_map

In [6]:
locations_map = image_paths_to_dict(paths_list)
f"Number of folders: {len(locations_map)}"

'Number of folders: 61'

In [7]:
# Sort images in locations_map by number in the filename
def extract_number(file_name):
    match = re.search(r'_(\d+)\.', file_name)
    if match:
        return int(match.group(1))
    return 0
    
for path in locations_map:
    locations_map[path] = sorted(locations_map[path], key=extract_number)

In [8]:
# Get most commmon offset between filenames in the list of paths
def get_most_common_offset(filenames):
    offsets = {}
    for i in range(len(filenames) - 1):
        current = extract_number(filenames[i])
        next = extract_number(filenames[i + 1])
        if (next - current) in offsets:
            offsets[(next - current)] += 1
        else:
            offsets[(next - current)] = 1

    max_offset = max(offsets, key = lambda x: offsets[x])
    return max_offset

In [9]:
# Split lists into multiple lists based on the most common offset 
def split_lists(locations_map):
    res_dict = {}
    for path in locations_map:
        offset = get_most_common_offset(locations_map[path])
        
        prev_number = extract_number(locations_map[path][0])
        main_list = []
        temp_list = [locations_map[path][0]]
        
        for i in range(1, len(locations_map[path])):
            order_number = extract_number(locations_map[path][i])
            if (order_number - prev_number) == offset:
                temp_list.append(locations_map[path][i])
            else:
                main_list.append(temp_list)
                temp_list = [locations_map[path][i]]
            prev_number = order_number
        if temp_list:
            main_list.append(temp_list)
        res_dict[path] = main_list
        
    return res_dict

In [10]:
splitted_locations_map = split_lists(locations_map)

In [47]:
# Get lists with sequential images 
def get_lists(splitted_locations_map):
    res = []
    for root_path in splitted_locations_map:
        for seq in splitted_locations_map[root_path]:
            if len(seq) > 1:
                res.append(list(map(lambda x: os.path.join(root_path, x), seq)))
    return res

In [48]:
sequential_paths = get_lists(splitted_locations_map)

In [49]:
def image_similarity(img1_path, img2_path):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    score, _ = structural_similarity(img1_gray, img2_gray, full=True)

    return score

In [50]:
def plot_neighboring_ssim(neighboring_similarities):
    indices = list(range(1, len(neighboring_similarities) + 1))
    plt.plot(indices, neighboring_similarities, marker='o', linestyle='-', color='b')
    plt.title('SSIM Index Plot')
    plt.xlabel('Image Pair Index')
    plt.ylabel('SSIM Value')
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()
    

In [51]:
def calculate_and_show_ssim(sequential_paths):
    all_similarities = []
    for j, seq in enumerate(sequential_paths):
        neighboring_similarities = []
        for i in range(len(seq) - 1):
            neighboring_similarities.append(image_similarity(seq[i], seq[i + 1]))
        all_similarities.append(neighboring_similarities)
        print(f"index: {j}; len: {len(seq)}; min ssim: {min(neighboring_similarities)}; avg ssim: {sum(neighboring_similarities) / len(neighboring_similarities)}")
    return all_similarities

In [ ]:
all_similarities = calculate_and_show_ssim(sequential_paths)

In [81]:
import pickle

In [83]:
with open('../data/sequential_images.pkl', 'wb') as f:
    pickle.dump(sequential_paths, f)

In [86]:
with open('../data/sequential_images.pkl', 'rb') as f:
    sequential_paths_loaded = pickle.load(f)